In [1]:
using ParallelTemperingMonteCarlo,BenchmarkTools,DelimitedFiles,MachineLearningPotential

Completed the MLP module based on the librunner.so file written by A. Knoll. Time to use this in PTMC simulations so I can finally generate results. This requires the librunnerjulia file and use of the MLP package.

First: establishing the struct.

This requires the symmetry functions, the nnp values itself, the g and f matrices used in the calculation and a dummy vector new_f_vec to preassign memory for later. I'd prefer to point to the nnp and symmetryfunction structs rather than save copies, I'll look into this later. 

In [2]:
struct  RuNNerPotential <: AbstractMachineLearningPotential
    nnp::NeuralNetworkPotential
    symmetryfunctions::Vector{AbstractSymmFunction}
    r_cut::Float64
end
function RuNNerPotential(nnp,symmetryfunction)
    r_cut = symmetryfunction[1].r_cut
    return RuNNerPotential(nnp,symmetryfunction,r_cut)
end


The challenge is potentials are not unique to each given state, meaning that g_matrix, f_matrix need to be part of the mc_state, not necessarily part of an immutable potential. They could be a vector of these matrices but this seems prohibitive. It's possible to assign them to mc_states? Have an NNP subtype?


In [3]:
mutable struct NNPState{T,N,BC}
    temp::T
    beta::T
    config::Config{N,BC,T}
    dist2_mat::Matrix{T}
    en_atom_vec::Vector{T}
    en_tot::T
    ham::Vector{T}
    max_displ::Vector{T}
    count_atom::Vector{Int}
    count_vol::Vector{Int}
    count_rot::Vector{Int}
    count_exc::Vector{Int}

    new_en_atom::Vector{T}

    g_matrix::Array{T}
    f_matrix::Matrix{T}
    new_dist2_vec::Vector{T}
    new_f_vec::Vector{T}
end


In [4]:
function NNPState(state::MCState, positions, n_atoms, symm_function_vector)

    f_matrix = cutoff_function.(sqrt.(state.dist2_mat),Ref(symm_function_vector[1].r_cut))

    g_matrix = total_symm_calc(positions,state.dist2_mat,f_matrix,symm_function_vector)

    nnstate = NNPState(
        state.temp, state.beta, state.config, state.dist2_mat, state.en_atom_vec, state.en_tot, state.ham, state.max_displ, state.count_atom, state.count_vol, state.count_rot, state.count_exc, g_matrix, f_matrix, zeros(n_atoms), zeros(n_atoms)
        )

    state = nnstate
    
    return state
end


NNPState

Functionally, this definition allows us to initialise the runner potential, the state and then use them to define the new potential specific state with the g_matrix included. 

In [5]:
# function energy_calculation(positions,dist2_mat,f_mat,total_symmetry_vector,neuralnetwork)
#     n_atoms = length(positions)
#     g_mat = total_symm_calc(positions,dist2_mat,f_mat,total_symmetry_vector)
#     return g_mat,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
# end
# function threnergy_calculation(positions,dist2_mat,f_mat,total_symmetry_vector,neuralnetwork)
#     n_atoms = length(positions)
#     g_mat = total_thr_symm_calc(positions,dist2_mat,f_mat,total_symmetry_vector)
#     return g_mat,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
# end

# function energy_calculation(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector,neuralnetwork)
#     n_atoms = length(positions)
#     g_mat = total_symm!(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector)
#     return g_matrix,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
# end

# function threnergy_calculation(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector,neuralnetwork)
#     n_atoms = length(positions)
#     g_mat = total_thr_symm!(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector)
#     return g_matrix,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
# end

In [6]:
function initial_energy_calculation(positions,dist2_mat,f_mat,runner_pot)
    n_atoms = length(positions)
    g_mat = total_symm_calc(positions,dist2_mat,f_mat,runner_pot.symmetryfunctions)
    return g_mat, sum(forward_pass(g_mat,n_atoms,runner_pot.nnp))
end
function initial_energy_calculation(positions,nnp_state,runner_pot)
    n_atoms = length(positions)

    nnp_state.g_matrix = total_symm_calc(positions,nnp_state.dist2_mat,nnp_state.f_matrix,runner_pot.symmetryfunctions)

    nnp_state.en_atom_vec = forward_pass(g_mat,n_atoms,runner_pot.nnp)

    nnp_state.en_tot = sum(nnp_state.en_atom_vec)

    return nnp_state
end

energy_calculation (generic function with 2 methods)

Critically, we require the get_energy function as defined in energyevaluation. Once this is done, we can simulate NNP's. 

Note-- This is done with the following input/output

input:  trial_positions,indices,mc_states,pot

the new positions, the new indices for them, the current states and the potential.

output:  energy_vector, dist2_new




In [1]:
function get_new_state_vars(trial_pos,atomindex,nnp_state,pot)
    nnp_state.new_dist2_vec = [distance2(pos,b,config.bc) for b in config.pos]
    nnp_state.new_dist2_vec[i_atom] = 0.
    
    nnp_state.new_f_vec = cutoff_function.(sqrt.(dist2_new),Ref(pot.r_cut))


    g_new = copy(nnp_state.g_matrix)

    g_new = total_symm!(g_new,nnp_state.config.pos,trial_pos,nnp_state.dist2_mat,nnp_state.new_dist2_vec,nnp_state.f_matrix,nnp_state.new_f_vec,atomindex,pot.symmetryfunctions)

    return g_new,nnp_state
end
function calc_new_energy(g_new,nnp_state,pot)
    new_en_vec = forward_pass(g_new,length(nnp_state.en_atom_vec),pot.nnp)

    return new_en_vec
end



In [7]:
function get_energy(trial_positions,indices,mc_states,pot::RuNNerPotential)

    
    return  #energyvector,mc_states
end